In [3]:
import chess
import chess.engine
import random
import numpy
from stockfish import Stockfish
import torch

def random_board(maxD=200):
    board = chess.Board()
    depth = random.randrange(0, maxD)
    
    for _ in range(depth):
        all_moves = list(board.legal_moves)
        random_move = random.choice(all_moves)
        board.push(random_move)
        if board.is_game_over():
            break
    return board


device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, conv_size, conv_depth):
        super(Net, self).__init__()
        # Adjust the in_channels of the first convolutional layer to match the input data
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=14 if i == 0 else conv_size,
                                              out_channels=conv_size,
                                              kernel_size=3,
                                              padding='same')
                                    for i in range(conv_depth)])
        self.fc1 = nn.Linear(conv_size * 8 * 8, 64)  # Adjust the size accordingly if the input volume changes
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, x):
        # Apply the convolutional layers
        for conv in self.convs:
            x = F.relu(conv(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Adjust `conv_size` and `conv_depth` as needed
conv_size = 64
conv_depth = 5

model = Net(conv_size, conv_depth) # the model definition must be exactly the same as the saved model

# Load the model state
model.load_state_dict(torch.load('model_weights.pth'))


model.to(device)

# Don't forget to switch to eval mode if you're doing inference
model.eval()

squares_index = {
    'a':0,
    'b':1,
    'c':2,
    'd':3,
    'e':4,
    'f':5,
    'g':6,
    'h':7,
}

def square_to_index(square):
    letter =  chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]

def split_dims(board):
    board3d = numpy.zeros((14,8,8), dtype=numpy.int8)
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = numpy.unravel_index(square, (8, 8))
            board3d[piece - 1][7 -idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = numpy. unravel_index(square, (8, 8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1
    # add attacks and valid moves too
    # so the network knows what is being attacked
    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j= square_to_index(move.to_square)
        board3d[12][i][j] = 1
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j= square_to_index(move.to_square)
        board3d[13][i][j] = 1
    board.turn = aux
    
    return board3d

In [8]:
def minimax_eval(board):
    board3d = split_dims(board)
    board3d = torch.tensor(numpy.expand_dims(board3d,0), dtype=torch.float32).to(device)
    return model(board3d)[0][0]

import chess

def lazy_move_ordering(board):
    captures = []
    checks = []
    others = []
    
    for move in board.legal_moves:
        if board.is_capture(move):
            captures.append(move)
        elif board.gives_check(move):
            checks.append(move)
        else:
            others.append(move)
    prioritized_moves = captures + checks + others
    return prioritized_moves



def minimax1(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over():
        return minimax_eval(board)
    
    if maximizing_player:
        max_eval = -numpy.inf
        for move in board.legal_moves:#order_moves(board):
            board.push(move)
            eval = minimax1(board, depth-1, alpha, beta, False)
            board.pop()
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = numpy.inf
        for move in board.legal_moves:
            board.push(move)
            eval = minimax1(board, depth-1, alpha, beta, True)
            board.pop()
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def minimax2(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over():
        return minimax_eval(board)
    
    if maximizing_player:
        max_eval = -numpy.inf
        for move in lazy_move_ordering(board):
            board.push(move)
            eval = minimax2(board, depth-1, alpha, beta, False)
            board.pop()
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = numpy.inf
        for move in board.legal_moves:
            board.push(move)
            eval = minimax2(board, depth-1, alpha, beta, True)
            board.pop()
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval
    
def get_ai_move1(board, depth):
    max_move = None
    max_eval = -numpy.inf
    
    for move in board.legal_moves:
        board.push(move)
        eval = minimax1(board, depth-1, -numpy.inf, numpy.inf, False)
        board.pop()
        if eval > max_eval:
            max_eval = eval
            max_move = move
        
    return max_move


def get_ai_move2(board, depth):
    max_move = None
    max_eval = -numpy.inf
    
    for move in board.legal_moves:
        board.push(move)
        eval = minimax2(board, depth-1, -numpy.inf, numpy.inf, False)
        board.pop()
        if eval > max_eval:
            max_eval = eval
            max_move = move
        
    return max_move

In [10]:
import time
dif = 0

for i in range(1):
    board =  random_board(50)
    st = time.time()
    get_ai_move1(board, 4)
    e1 = st- time.time()
    
    st = time.time()
    get_ai_move1(board, 4)
    e2 = st- time.time()
    
    dif += e1-e2
    
    print(dif)
    
    
    

49.60081148147583
